# Using the Similarity Model 

This notebook allows you to try out the model by submitting an image and getting back the top 10 most similar images.  

First we'll set up the workspace with some imports and a few new functions.

In [ ]:
%matplotlib inline
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from skimage import data
import pandas as pd
from revoscalepy import RxSqlServerData, rx_data_step, rx_import
from parameters import *
from run_image_similarity import rank_candidates
from image_utils import show_img, submit_img

# Submiting a New Image 
This notebook is set up to process one image and return the 10 most similar images.  Try it out with a few of the images below by commenting out all but one of the images. 

In [ ]:
img = "../data/dotted/81.jpg"
#img = "../data/leopard/147.jpg"
#img = "../data/striped/379.jpg"
show_img(img, "Image to rank")

## Create the `TABLE_QUERY` table
This step will wipe out the previous table contents and add our image into the table which will then be processed below.

In [ ]:
conn_str = "DRIVER={SQL Server};SERVER=localhost;PORT=1433;DATABASE=ImageSimilarity_Py;TRUSTED_CONNECTION=True"
submit_img(conn_str, TABLE_QUERY, img )

## Featurize and predict scores for new image
1.  We assume the predicted scores of all the candidate images are already saved in SQL table `@scores_table` (which they will be if you have run the PowerShell to create this solution). The first step in finding similar images is to then load the predicted scores for all the candidate images from the SQL table. We don't need to calculate them in this step.

2. Our new image is in the the `TABLE_QUERY` table.  We load the query image from the SQL table, and then featurize the query image using pre-trained Resnet. Note this has to be the same pre-trained model which used in the step 1.

3. Load the model which trained in step 3 from the SQL table `TABLE_MODEL`, and calculate the predicted scores for the query image using the model.

4. Calculate the Cosine distance between each query image and all the candidates, based on the distance, return top K similar images for each query images. Users can set up parameter `TOP_K_CANDIDATES` to decide how many similar images should be returned for each query image.
 For example, here we set `TOP_K_CANDIDATES` equal to 10, so in the result table `TABLE_RESULTS`, each query image has 10 similar images.

(*Parameters  `TABLE_QUERY`,`TABLE_MODEL`, `TOP_K_CANDIDATES`, and `TABLE_RESULTS` are set in the `parameters` file and imported at the top of this notebook.*)

In [ ]:
rank_candidates(conn_str, TABLE_QUERY, TABLE_MODEL, TABLE_RESULTS)

## Retrieve results
We will now read the reults from the above scoring.

In [ ]:
data_source = RxSqlServerData(table=TABLE_RESULTS, connection_string=conn_str)
df = pd.DataFrame(rx_import(input_data = data_source)) 
print("retrieved similar images")

## Display results
Now we're ready to show the images.  Since we know we have 10 results, we'll display them in a 2x5 grid. 

In [ ]:
# display the original image
show_img(img, 'Image to rank')

i = 0
plt.figure(figsize=(20,10))

# loop through the top 10 similar images and display in a grid
while i < len(df):
    photo_data = misc.imread(df['refImage'][i])
    plt.suptitle('Similar Images', fontsize=20)
    plt.subplot(2,5,i+1)
    plt.axis('off')
    plt.imshow(photo_data)
    i += 1
plt.show